In [1]:
from huggingface_hub import hf_hub_download

ModuleNotFoundError: No module named 'huggingface_hub'

In [5]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="kyegorov/mcd_rppg",local_dir='./mcd_rppg',cache_dir='./mcd_rppg/cache', repo_type="dataset")

Fetching 13204 files:   9%|▊         | 1142/13204 [05:50<2:18:00,  1.46it/s]Error while downloading from https://huggingface.co/datasets/kyegorov/mcd_rppg/resolve/5ccaab9abaa45b99783cdab1ee4c05011ea64a5a/ecg/9554_before.json: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out.
Trying to resume download...
Fetching 13204 files:  73%|███████▎  | 9672/13204 [23:34<7:30:26,  7.65s/it] Error while downloading from https://cdn-lfs-us-1.hf.co/repos/57/4b/574b5996cc0fc37491a94327a8f8ea8f4e430e33d5fc5b9bac83946cbc3db74c/79d1ecbeb722bf7bd8da8beeb83feb99dfa865f9e930a29a7ad96aed2be14b61?response-content-disposition=inline%3B+filename*%3DUTF-8%27%271234_FullHDwebcam_after.avi%3B+filename%3D%221234_FullHDwebcam_after.avi%22%3B&response-content-type=video%2Fx-msvideo&Expires=1729400090&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTQwMDA5MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzU3LzRiLzU3NGI1OTk2Y2MwZmMzNzQ5MWE5N

'C:\\Users\\Ivan\\Desktop\\Proj\\rppg\\mcd_rppg'

In [5]:
import os


os.getcwd()

'e:\\FirstProj\\rPPG'

In [2]:
dir(rPPG_Toolbox.dataset)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'data_loader']

In [3]:
parser = rPPG_Toolbox.dataset.data_loader.BaseLoader

In [4]:
import numpy as np
from scipy.stats import gaussian_kde

In [14]:
np.matrix(xy)

matrix([[109.4941901 ,  88.8418154 ],
        [110.06596332,  92.24078273]])

In [19]:
xy = np.array([[109.4941901 ,  88.8418154 ],
       [110.06596332,  92.24078273]]).astype(np.float64)
#xy = np.matrix(xy)
gaussian_kde(xy)(xy)

array([52103.26720011, 52103.26720011])

In [21]:
np.linalg.eigh(np.cov(xy))[0]

array([1.42108547e-14, 3.72128822e+02])

In [23]:
np.eye(np.cov(xy))

TypeError: only integer scalar arrays can be converted to a scalar index